In [1]:
from model import *
from utilities import *

In [2]:
extracted_features = np.load("./checkpoints/extracted_features_minmax.npy")
labels = np.load("./checkpoints/labels.npy")

In [3]:
from sklearn.model_selection import train_test_split
train_X,test_X,train_y,test_y = train_test_split(extracted_features,labels,test_size=0.1,random_state=7)

In [4]:
train_X.shape,test_X.shape

((53, 100), (6, 100))

In [5]:
train_dataset = FeatureDataset(train_X,train_y)
test_dataset = FeatureDataset(test_X,test_y)

In [6]:
set_seed(666)
train_loader = DataLoader(
    train_dataset,
    batch_size=CFG.batch_size,
    shuffle=True,
    drop_last=True,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=6,
    shuffle=False)

# model 调用和训练

In [7]:
model = CustomModel()

In [8]:
jj = next(iter(train_loader))
model(jj["input"])

tensor([[-0.6722, -0.7145],
        [-0.1946, -1.7326],
        [-0.2036, -1.6917],
        [-0.1866, -1.7706],
        [-0.8918, -0.5275],
        [-1.0487, -0.4314],
        [-1.1027, -0.4034],
        [-0.1823, -1.7917],
        [-1.4367, -0.2714],
        [-1.0524, -0.4294]], grad_fn=<LogSoftmaxBackward0>)

In [11]:
def fit_model(train_loader,test_loader,path,RESUME=False):
    model=CustomModel()
#     optimizer = getattr(torch.optim, CFG.optimizer)(filter(lambda p: p.requires_grad, model.parameters()), 
#                                                     lr=0.001,weight_decay=0.0001)  # 优化器
    optimizer = torch.optim.SGD(model.parameters(), lr = CFG.lr,weight_decay = CFG.weight_decay)
#     scheduler = getattr(torch.optim.lr_scheduler, CFG.scheduler)(optimizer, gamma=CFG.sc_Gamma)  # 指数型学习率
    lsrloss = LSR(CFG.num_classes,0.05)
    start_epoch=-1
    if RESUME:
        path_checkpoint = path+'/ckpt_aml.pth'   # 断点路径
        checkpoint = torch.load(path_checkpoint)  # 加载断点
        model.load_state_dict(checkpoint['net'])  # 加载模型可学习参数
        optimizer.load_state_dict(checkpoint['optimizer'])  # 加载优化器参数
        start_epoch = checkpoint['epoch']  # 设置开始的epoch
        #scheduler.load_state_dict(checkpoint['scheduler'])
#     batch_iterator = zip(loop_iterable(train_loader_sample), 
#                      loop_iterable(train_loader_batch))
#     n_batches = max(len(train_loader_sample), len(train_loader_batch))

    for epoch in range(start_epoch+1,CFG.epoches):
        for data in train_loader:
            input_feature = data["input"]
            true_label = data["label"]
            pred_label = model(input_feature)
            loss=lsrloss(pred_label,true_label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #scheduler.step()
        
        with torch.no_grad():
            val_loss_sum = 0
            for test_data in test_loader:
                input_feature_test = test_data["input"]
                true_label_test = test_data["label"]
                pred_label_test = model(input_feature_test)
                val_loss=lsrloss(pred_label_test,true_label_test)

                val_loss_sum += val_loss
            val_loss_mean = val_loss_sum/(len(test_loader))
        #print(shot_RUL[0:10],RUL_pred_val[0:10])

                
        if  epoch % 20 == 0:
            print(f"epoch:{epoch}, loss:{loss.item()},val_loss:{val_loss_mean.item()}")#,lr:{optimizer.state_dict()['param_groups'][0]['lr']}")
            checkpoint = {
                "net": model.state_dict(),
                'optimizer': optimizer.state_dict(),
                "epoch": epoch,
                #'scheduler': scheduler.state_dict()
            }
            if not os.path.isdir(path):
                os.mkdir(path)
            torch.save(checkpoint,
                       path+'/ckpt_aml.pth')
#         if val_loss_mean.item() < 1600:
#             print(f"epoch:{epoch}, loss:{loss.item()},val_loss:{val_loss_mean.item()}")      
#                   #lr:{optimizer.state_dict()['param_groups'][0]['lr']}")
#             save_model_weights(model, "model_his_aml.pt",
#                                cp_folder=path)
#             break
        elif epoch == (CFG.epoches-1):
            print(f"epoch:{epoch}, loss:{loss.item()}",)
            save_model_weights(model, "model_his_aml.pt",
                               cp_folder=path)

In [12]:
fit_model(train_loader,test_loader,"./checkpoints",RESUME=False)

epoch:0, loss:0.7185090780258179,val_loss:0.6831567287445068
epoch:20, loss:0.3153533339500427,val_loss:0.2991580665111542
epoch:40, loss:0.2304818332195282,val_loss:0.2352849245071411
epoch:60, loss:0.21511027216911316,val_loss:0.21655400097370148
epoch:80, loss:0.21765132248401642,val_loss:0.20905137062072754
epoch:100, loss:0.20517262816429138,val_loss:0.20562972128391266
epoch:120, loss:0.2072899043560028,val_loss:0.2037397176027298
epoch:140, loss:0.2053045779466629,val_loss:0.20265112817287445
epoch:160, loss:0.20664076507091522,val_loss:0.20201390981674194
epoch:180, loss:0.205412819981575,val_loss:0.2015949934720993
epoch:199, loss:0.20561985671520233

 -> Saving weights to ./checkpoints\model_his_aml.pt



# 模型的调用

In [14]:
aml_model=CustomModel()
aml_model.load_state_dict(torch.load("./checkpoints/model_his_aml.pt"))

<All keys matched successfully>

In [15]:
test_ = next(iter(test_loader))

In [17]:
pred = torch.exp(aml_model(test_["input"]))

In [19]:
pred,test_["label"]

(tensor([[0.0415, 0.9585],
         [0.0627, 0.9373],
         [0.0567, 0.9433],
         [0.9474, 0.0526],
         [0.9159, 0.0841],
         [0.0304, 0.9696]], grad_fn=<ExpBackward0>),
 tensor([[0., 1.],
         [0., 1.],
         [0., 1.],
         [1., 0.],
         [1., 0.],
         [0., 1.]]))